# LIDAR Point Cloud Visualization

**Learning Objective**: Understand LIDAR sensor output as 3D point clouds

**Estimated Time**: 12 minutes

This notebook demonstrates how LIDAR sensors (like Velodyne VLP-16) create 3D representations of environments.

In [ ]:
# Install dependencies
!pip install numpy==1.24.0 matplotlib==3.7.0

In [ ]:
from typing import Tuple
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

np.random.seed(42)

In [ ]:
def simulate_lidar_scan(
    num_points: int = 1000,
    max_range: float = 10.0,
    vertical_fov: Tuple[float, float] = (-15, 15)
) -> np.ndarray:
    """
    Simulate a LIDAR scan of a simple environment (walls and objects).
    """
    points = []
    
    # Simulate walls
    for _ in range(num_points // 2):
        azimuth = np.random.uniform(0, 2 * np.pi)
        elevation = np.deg2rad(np.random.uniform(vertical_fov[0], vertical_fov[1]))
        
        if np.cos(azimuth) > 0:
            distance = min(5.0 / abs(np.cos(azimuth)), max_range)
        else:
            distance = min(5.0 / abs(np.cos(azimuth)) if np.cos(azimuth) != 0 else max_range, max_range)
        
        x = distance * np.cos(elevation) * np.cos(azimuth)
        y = distance * np.cos(elevation) * np.sin(azimuth)
        z = distance * np.sin(elevation)
        
        points.append([x, y, z])
    
    # Simulate obstacles
    for _ in range(num_points // 2):
        x = np.random.uniform(1.5, 2.5)
        y = np.random.uniform(1.5, 2.5)
        z = np.random.uniform(-0.5, 0.5)
        points.append([x, y, z])
    
    return np.array(points)

def add_lidar_noise(
    points: np.ndarray,
    range_accuracy: float = 0.03
) -> np.ndarray:
    """
    Add realistic noise to LIDAR measurements (±3cm for Velodyne VLP-16).
    """
    noise = np.random.normal(0, range_accuracy, size=points.shape)
    return points + noise

def calculate_point_cloud_stats(points: np.ndarray) -> dict:
    """Calculate statistics about the point cloud."""
    return {
        'num_points': len(points),
        'mean_distance': np.mean(np.linalg.norm(points, axis=1)),
        'max_distance': np.max(np.linalg.norm(points, axis=1)),
        'height_range': np.ptp(points[:, 2]),
        'density': len(points) / (np.ptp(points[:, 0]) * np.ptp(points[:, 1]))
    }

In [ ]:
# Simulate LIDAR scan
points = simulate_lidar_scan(num_points=2000, max_range=10.0)
noisy_points = add_lidar_noise(points, range_accuracy=0.03)

# Calculate statistics
stats = calculate_point_cloud_stats(noisy_points)

print("=" * 60)
print("LIDAR POINT CLOUD VISUALIZATION")
print("=" * 60)
print("\nPoint Cloud Statistics:")
print(f"  • Total Points: {stats['num_points']:.0f}")
print(f"  • Mean Distance: {stats['mean_distance']:.2f} m")
print(f"  • Max Distance: {stats['max_distance']:.2f} m")
print(f"  • Height Range: {stats['height_range']:.2f} m")
print(f"  • Point Density: {stats['density']:.1f} points/m²")
print("\n" + "=" * 60)
print("KEY INSIGHT: LIDAR provides sparse 3D structure")
print("Robots must process ~300,000 points/second in real-time!")
print("=" * 60)

In [ ]:
# Visualize 3D point cloud
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

colors = noisy_points[:, 2]  # Color by height

scatter = ax.scatter(
    noisy_points[:, 0],
    noisy_points[:, 1],
    noisy_points[:, 2],
    c=colors,
    cmap='viridis',
    s=2,
    alpha=0.6
)

ax.set_xlabel('X (meters)', fontsize=11)
ax.set_ylabel('Y (meters)', fontsize=11)
ax.set_zlabel('Z (meters)', fontsize=11)
ax.set_title('LIDAR Scan: Simulated Room with Obstacle', fontsize=14, fontweight='bold')

cbar = plt.colorbar(scatter, ax=ax, pad=0.1, shrink=0.8)
cbar.set_label('Height (m)', fontsize=10)

max_range = np.abs(noisy_points).max()
ax.set_xlim([-max_range, max_range])
ax.set_ylim([-max_range, max_range])
ax.set_zlim([-max_range/2, max_range/2])

plt.tight_layout()
plt.show()

## Expected Output

You should see:
1. **Statistics**: ~2000 points, mean distance 4-6m
2. **3D Visualization**: Colored points showing walls and a box obstacle

**Key Takeaway**: LIDAR provides 3D structure but is sparse. Robots process hundreds of thousands of points per second for navigation and obstacle avoidance.